# 5.3.1 The Validation Set Approach

In this section, we’ll explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the Auto data set.

After loading the data, we'll use the `radom_state` parameter in the `train_test_split()` function from the `sklearn` pacakge in order to set a seed for random number generator, so that you’ll obtain precisely the same results as those shown in the textbook. It is generally a good idea to set a random seed when performing an analysis such as cross-validation that contains an element of randomness, so that the results obtained can be reproduced precisely at a later time.

We begin by using the `train_test_split()` functions to split the set of observations into two halves. We’ll start by selecting a random subset of 196 observations out of the original 392 observations. We refer to these observations as the training set.

In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split

autoData = pd.read_csv("../../data/Auto.csv", index_col = None, na_values = ['?'])  # data contains ? for NA values
autoData = autoData.dropna() # remove NA since for regression

train, test = train_test_split(autoData, test_size = 0.5, random_state = 1)

We then make a `LinearRegression` object to access methods from the `sklearn` package in SciPy to fit a linear regression using only the observations corresponding to the training set.

In [115]:
from sklearn.linear_model import LinearRegression

model_LR = LinearRegression()
model_LR.fit(train[['horsepower']], train[['mpg']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

We now use the `predict` method to estimate the response for the test observations, and we use the `mean()` function to calculate the MSE of the 196 observations in the test set:

In [116]:
((model_LR.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    24.802121
dtype: float64

Therefore, the estimated test MSE for the linear regression fit is `24.80`.  We can now create an object to perfom a quadradic and cubic regression by utilzing the `PolynomialFeatures` and `Pipeline` packages from `sklearn`.

In [117]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

First on a quadradic

In [118]:
model_QUAD = Pipeline([('poly', PolynomialFeatures(degree = 2)),
                       ('linear', LinearRegression(fit_intercept = False))]) # create a polynomial object
model_QUAD.fit(train[['horsepower']], train[['mpg']])

((model_QUAD.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    18.848293
dtype: float64

Then a cubic model

In [119]:
model_CUBIC = Pipeline([('poly', PolynomialFeatures(degree = 3)),
                       ('linear', LinearRegression(fit_intercept = False))])
model_CUBIC.fit(train[['horsepower']], train[['mpg']])

((model_CUBIC.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    18.805111
dtype: float64

These error rates are `18.85` and `18.81`, respectively. If we choose a different training set instead, then we will obtain somewhat different errors on the validation set. We can test this out by setting a different random seed:

In [120]:
train, test = train_test_split(autoData, test_size = 0.5, random_state = 2)

In [121]:
model_LR2 = LinearRegression()
model_LR2.fit(train[['horsepower']], train[['mpg']])

((model_LR2.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    23.442644
dtype: float64

In [122]:
model_QUAD2 = Pipeline([('poly', PolynomialFeatures(degree = 2)),
                       ('linear', LinearRegression(fit_intercept = False))])
model_QUAD2.fit(train[['horsepower']], train[['mpg']])

((model_QUAD2.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    18.550199
dtype: float64

In [123]:
model_CUBIC2 = Pipeline([('poly', PolynomialFeatures(degree = 3)),
                       ('linear', LinearRegression(fit_intercept = False))])
model_CUBIC2.fit(train[['horsepower']], train[['mpg']])

((model_CUBIC2.predict(test[['horsepower']]) - test[['mpg']]) ** 2).mean()

mpg    18.595222
dtype: float64

Using this split of the observations into a training set and a validation set, we find that the validation set error rates for the models with linear, quadratic, and cubic terms are `23.44`, `18.55`, and `18.60`, respectively.

These results are consistent with our previous findings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favor of a model that uses a cubic function of horsepower.

# 5.3.2 Leave-One-Out Cross-Validation

The LOOCV estimate can be automatically computed for any generalized linear model using the `cross_val_score()` and `cross_val_predict()` functions from the `sklearn` package.  These two functions can be tuned for various sampling and scoring methods depending on your needs.  For the example below, we will use the `cross_val_predict()` function to perform LOOCV, obtaining prediction values and then compute the mean square erorr of the LOOCV using the `mean_square_error()` function also from `sklearn`.

First import the packages

In [124]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

Then perform the cross validation using the `cross_val_predict()` function.  For LOOCV we will first pass the function our `LinearRegression` model object from above, set the `cv` paramater equal to the number of observations in our dataset for LOOCV, and then use `mean_square_error()` function to compute a cross validation score.

In [125]:
predict = cross_val_predict(model_LR, autoData[['horsepower']],
                         autoData[['mpg']],
                         cv = len(autoData[['mpg']]))

mean_squared_error(autoData[['mpg']], predict)

24.23151351792923

In this case the prediction error is `24.23`.  This isn't too useful unless it is used to compare several different models which we will do below.

We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we use the for loop which iteratively fits polynomial regressions for polynomials of order 1 to 5 and computes the associated cross-validation error.

This command may take a couple of minutes to run.

In [126]:
model_scores = [];
for i in range(1, 6):
    model_GLR = Pipeline([('poly', PolynomialFeatures(degree = i)),
                          ('linear', LinearRegression(fit_intercept = False))])
    model_GLR.fit(autoData[['horsepower']], autoData[['mpg']])
    
    pred = cross_val_predict(model_GLR, autoData[['horsepower']],
                             autoData[['mpg']],
                             cv = len(autoData[['horsepower']]))
    
    model_scores.append(mean_squared_error(autoData[['mpg']], pred))
    
model_scores

[24.231513517929233,
 19.248213124490388,
 19.334984064020858,
 19.424430299871521,
 19.03326050458827]

Here we see a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement from using higher-order polynomials.

# 5.3.3 k-Fold Cross-Validation

The `cross_val_predict()` and `cross_val_score()` functions can also be used to implement k-Fold CV. Above, we set the `cv` parameter equal to the number of observations in order to perform LOOCV.  Below we use k = 10, a common choice for k, on the Auto data set. We once again set a random seed and initialize a vector in which we will store the CV errors corresponding to the polynomial fits of orders one to ten.

In [127]:
model_scores = [];
for i in range(1, 11):
    model_GLR = Pipeline([('poly', PolynomialFeatures(degree = i)),
                          ('linear', LinearRegression(fit_intercept = False))])
    model_GLR.fit(autoData[['horsepower']], autoData[['mpg']])
    
    pred = cross_val_predict(model_GLR, autoData[['horsepower']],
                             autoData[['mpg']],
                             cv = 10)
    
    model_scores.append(mean_squared_error(autoData[['mpg']], pred))
    
model_scores

[27.416194818355972,
 21.202293642897722,
 21.302479720296542,
 21.319376801860198,
 20.869201167625281,
 20.865450400076767,
 20.918115351773562,
 25.610794730521821,
 42.800730506916601,
 84.280038134738064]

Notice that the computation time is much shorter than that of LOOCV.  We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## An Application to Default Data

Now that you’re armed with more useful technique for resampling your data, let’s try fitting a model for the Default dataset.  Before fitting, we'll need to import the data and convert to dummy variables in order to use the functions in the `sklearn` package.

In [128]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

defaultData = pd.read_csv('../../data/Default.csv', index_col = 0)

enc = LabelEncoder()
defaultData.default = enc.fit_transform(defaultData.default)
defaultData.student = enc.fit_transform(defaultData.student)

First we’ll try just holding out a random 20% of the data:

In [129]:
cMatrix_LG = []
score_LG = []

for i in range(0, 10):
    train, test = train_test_split(defaultData, test_size = 0.2, random_state = i)
    
    # Fit the logistic model
    model_LG = LogisticRegression()
    model_LG.fit(train[['balance', 'student']], train['default'])
    
    # Use the model to predict the response
    pred_LG = model_LG.predict(test[['balance', 'student']])
    
    # Confusion matrix
    cMatrix_LG.append(confusion_matrix(test['default'], pred_LG))
    
    # Prediction Scores
    score_LG.append(model_LG.score(test[['balance', 'student']], test['default']))

Now print the results:

In [130]:
for i in range(0, len(cMatrix_LG)):
    print(cMatrix_LG[i])

[[1924    2]
 [  56   18]]
[[1932    9]
 [  43   16]]
[[1933   10]
 [  43   14]]
[[1939    2]
 [  39   20]]
[[1928    3]
 [  47   22]]
[[1930    4]
 [  49   17]]
[[1931    3]
 [  52   14]]
[[1929    4]
 [  47   20]]
[[1933    2]
 [  49   16]]
[[1944    3]
 [  36   17]]


In [131]:
score_LG

[0.97099999999999997,
 0.97399999999999998,
 0.97350000000000003,
 0.97950000000000004,
 0.97499999999999998,
 0.97350000000000003,
 0.97250000000000003,
 0.97450000000000003,
 0.97450000000000003,
 0.98050000000000004]

Our accuracy is really high on this data, but we’re getting different error rates depending on how we choose our test set. That’s no good!

Unfortunately this dataset is too big for us to run LOOCV, so we’ll have to settle for k-fold. In the space below, build a logistic model on the full `Default` dataset and then run 5-fold cross-validation to get a more accurate estimate of your test error rate:

In [132]:
# Your code here

# 5.3.4 The Bootstrap

We illustrate the use of the bootstrap in the simple example of Section 5.2, as well as on an example involving estimating the accuracy of the linear regression model on the `Auto` data set.

## Estimating the Accuracy of a Statistic of Interest

One of the great advantages of the bootstrap approach is that it can be applied in almost all situations. No complicated mathematical calculations are required. Performing a bootstrap analysis in Pyhton entails writing a function that will randomly sample you data, with or without replacement, and compute the statistics of interest.

The `Portfolio` data set in the `ISLR` package is described in Section 5.2. To illustrate the use of the bootstrap on this data, we must first create a function, `alpha_boot()`, which takes as input the `(X,Y)` data as well as the number of times to resample and compute α. The function then outputs the estimates for α based on the selected observations.

First we will read in the `Portfolio` data and import the `numpy` package as we will need some of its functions below.

In [133]:
import numpy as np
dataPort = pd.read_csv('../../data/Portfolio.csv', index_col = 0)

In [134]:
def alpha_boot(data, n):
    alpha = [] # empty array to hold statistics from each iteration
    for i in range(0, n):
        dataSample = data.sample(len(data), replace = True) #sample data with replacement
        x = dataSample['X']
        y = dataSample['Y']
        alpha.append((np.var(y) - np.cov(x, y)[0, 1]) / (np.var(x) + np.var(y)- 2 * np.cov(x, y)[0, 1]))
    return(alpha)

Now we can run the `alpha_boot()` function for 1000 iterations with a sample size of 100 and compute the mean.

In [135]:
alpha1 = alpha_boot(dataPort, 1000)
np.mean(alpha1)

0.58207219281140044

Also of interest may be the standard deviation.

In [136]:
np.std(alpha1)

0.090831910379228983

## Estimating the Accuracy of a Linear Regression Model

The bootstrap approach can be used to assess the variability of the coefficient estimates and predictions from a statistical learning method. Here we use the bootstrap approach in order to assess the goodness of fit for the regression model to predict `mpg` as a function of `horsepower` in the `Auto` data set.

We first create a simple function, `bootstrap_reg()`, which takes the data, number of bootstrap samples to take, and the degree of the polynomial to fit, and returns the square errors for each sample.

In [137]:
def bootstrap_reg(data, n, deg):
    error = []
    for i in range(0, n):
        model = Pipeline([('poly', PolynomialFeatures(degree = deg)),
                          ('linear', LinearRegression(fit_intercept = False))])
        dataSample = data.sample(len(data), replace = True)
        model.fit(dataSample[[0]], dataSample[[1]])
        error.append(((dataSample[[1]] - model.predict(dataSample[[0]])) ** 2).mean()[0])
    return(error)

Now let's use the function to perform a bootstrap on the `Auto` data set for a simple linear regression case.  Once we have the errors, we can compute some statistics to determine the quality of fit the same as above.

In [141]:
import numpy as np
bs_LR = bootstrap_reg(autoData, 1000, 1)

In [142]:
np.mean(bs_LR)

1.1429063902156222

In [143]:
np.std(bs_LR)

0.082023136952851947

Recall that we found previously that the relationship between `horsepower` and `mpg` is better characterized by a quadratic model. Let’s see how the error rates compare when we fit that instead of a linear model:

In [144]:
bs_QUAD = bootstrap_reg(autoData, 1000, 2)

In [145]:
np.mean(bs_QUAD)

0.75115009414858103

In [146]:
np.std(bs_QUAD)

0.076018747428914638

As before, a quadradic provides a bit better fit than the linear model alone.

To get credit for this lab, please post your answers to the following questions: - How did the cross-validated error rate compare to the models where you held out a validation set? Why do you think that is? - How do you explain the discrepancy between the bootstrap evaluation and the standard error evaluation of the linear model predicting `mpg` from `horsepower`? - What was the most confusing part of today’s class?